# Aim

Solve an LQR toy problem to see what to do

In [1]:
import numpy as np
from scipy import linalg as la
from scipy import stats as st
from matplotlib import pyplot as plt

inv = np.linalg.inv
mvn = st.multivariate_normal
rms = lambda data: round(np.sqrt(np.mean((data - np.mean(data)) ** 2)), 4)

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
from src.controllers import KalmanLQG

Running in simulation mode.
Recomputed interaction matrix and command matrix


/Users/adityasengupta/research/ao/set-tt-control/notebooks/../src/optics/image.py:109: UserWarning: Exposure time in sim optics is not used yet.
  warnings.warn("Exposure time in sim optics is not used yet.")
/Users/adityasengupta/research/ao/set-tt-control/notebooks/../src/optics/image.py:117: UserWarning: Image propagation from the DM has not been implemented.
  warnings.warn("Image propagation from the DM has not been implemented.")


In [25]:
s = 2 # state
m = 2 # measure
p = 1 # input
nsteps = 1000

In [53]:
A = np.array([[0.94154541, -0.97488067], [1, 0]])
B = np.random.randn(s, p)
C = np.random.randn(m, s)
Q = np.eye(s)
R = 1e-4 * np.eye(p)
W = 0.01 * np.eye(s)
V = 0.0036 * np.eye(m)
klv = KalmanLQG(A, B, C, W, V, Q, R)
process_dist = mvn(cov=W, allow_singular=True)
measure_dist = mvn(cov=V, allow_singular=True)
x = process_dist.rvs()

Solved discrete ARE.
Solved discrete ARE.


In [54]:
states_un = np.zeros((nsteps, s))
states_un[0,:] = x
for i in range(1, nsteps):
    states_un[i,:] = A @ states_un[i-1,:] + process_dist.rvs()
    
rms(states_un @ C.T)

0.5469

In [55]:
states_nokf = np.zeros((nsteps, s))
states_nokf[0,:] = x
for i in range(1, nsteps):
    states_nokf[i,:] = (A + B @ klv.L) @ states_nokf[i-1,:] + process_dist.rvs()
    
rms(states_nokf @ C.T)

0.0745

In [56]:
states = np.zeros((nsteps, s))
states[0,:] = x
klv.x = x
for i in range(1, nsteps):
    u = klv.control()
    klv.predict(u)
    x = A @ states[i-1,:] + B @ u + process_dist.rvs()
    y = C @ x + measure_dist.rvs()
    klv.update(y)
    if i % 10 == 0:
        print(x, klv.x)
    states[i,:] = x
    
rms(states @ C.T)

[ 0.08970746 -0.07743208] [0.14014318 0.02533453]
[0.12122417 0.12165426] [0.11879893 0.13802522]
[0.2015712  0.16831747] [0.28861957 0.13465835]
[-0.41495774 -0.09880133] [-0.24613697 -0.13450642]
[-0.10924285  0.03370537] [ 0.0494415  -0.05752534]
[-0.04704582  0.04393269] [ 0.02417784 -0.04477642]
[-0.14250427 -0.11344281] [-0.14997493 -0.11649358]
[-0.05968233 -0.08201909] [ 0.02614824 -0.11651678]
[-0.13613166  0.36413894] [-0.00906005  0.34146346]
[0.07212887 0.15414943] [-0.06965365  0.19681412]
[ 0.03064003 -0.13119475] [ 0.04133081 -0.1063541 ]
[-0.24347224  0.26889333] [-0.12619976  0.19605247]
[-0.01316569 -0.15858753] [-0.07107563 -0.08597198]
[-0.15847134 -0.18705721] [-0.03826207 -0.14516136]
[0.07495897 0.03766318] [0.07197412 0.02597043]
[-0.01800144 -0.1422081 ] [ 0.02017455 -0.1890905 ]
[0.09951851 0.22177794] [0.17143633 0.10360605]
[-0.054456    0.27288269] [-0.04978896  0.24379975]
[-0.00046729  0.29474141] [0.13763885 0.1997485 ]
[-0.19015626 -0.11296722] [-0.0145

0.1397